In [1]:
from result_saver import SaverProvider
provider = SaverProvider()

In [2]:
# parameter grid
distances = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
devices = ["ibm_sherbrooke", "ibm_brisbane", "ibm_nazca", "ibm_cusco"]
logicals = ['0', '1']

In [4]:
from tqdm import tqdm

from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit import transpile

from soft_info import get_repcode_layout
from Scratch import metadata_helper

# innefficent because most retrieve provider not needed
for distance in tqdm(distances):    
    for device in devices:
        for logical in logicals:
            code = RepetitionCodeCircuit(distance, distance)
            qc = code.circuit[logical]

            backend = provider.get_backend(device)
            layout = get_repcode_layout(distance, backend, _is_hex=True, plot=False)
            transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=3)

            n_shots = 1e6 / distance**2
            metadata = metadata_helper(descr="Run bigger Repetition codes" ,code="RepetitionCodeCircuit", distance=distance, rounds=distance, logical=logical, layout='_is_hex=True')
            backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single') # meas_level=1 for IQ  
            


  0%|          | 0/10 [00:00<?, ?it/s]